# Part 1

The following code represents a simple, inefficient pattern search that might be found during EDA in a jupyter notebook.

In a separate `.py` file, refactor this code using OOP to make it more extensible and efficient.

Your object should be able to do each of the following:

1. Accept either a string or a list of words as input text to be searched for pattern matches (if a string is passed, only words in the string that match the pattern should be caught)
2. Allow for registering an arbitrary number of pattern groups to search input text
3. Allow the user to specify a specific pattern group to search the text for matches
4. Allow the user to search input text with all registered patterns and return all matches by pattern group

In [3]:
import pandas as pd
import time

### Naive Implementation

In [5]:
#my_words = ['dog', 'cat', 'mouse', 'bird', 'snake', 'kitten', 'puppy', 'rat', 'cat']
my_words = test

pattern_1_words = []
pattern_2_words = []

pattern_group_1 = [
    'at',
    'ou'
]

pattern_group_2 = [
    'in',
    'mo'
]

start = time.time()
for word in my_words:
    for pat in pattern_group_1:
        if pat in word:
            pattern_1_words.append(word)
print(time.time() - start)

start = time.time()
for word in my_words:
    for pat in pattern_group_2:
        if pat in word:
            pattern_2_words.append(word)
print(time.time() - start)

print(f"Pattern 1: {pattern_1_words}")
print(f"Pattern 2: {pattern_2_words}")

0.0068280696868896484
0.005951881408691406
Pattern 1: ['States', 'at', 'whatsoever.', 'You', 'at', 'you', 'located', 'States,', 'you', 'country', 'you', 'located', 'Date:', 'updated:', 'Caterpillar', 'Croquet-Ground', 'conversations', 'what', 'thought', '“without', 'conversations?”', 'could,', 'would', 'trouble', 'that;', 'out', 'late!”', 'thought', 'that', 'ought', 'at', 'at', 'natural);', 'watch', 'out', 'waistcoat-pocket,', 'at', 'that', 'waistcoat-pocket,', 'watch', 'out', 'fortunately', 'out', 'that', 'about', 'found', 'about', 'what', 'out', 'what', 'at', 'that', 'great', 'underneath,', 'thought', 'at', 'wouldn’t', 'about', 'house!”', 'Would', 'aloud.', 'that', 'would', 'four', 'thousand', 'you', 'though', 'that’s', 'about', 'what', 'Latitude', 'what', 'Latitude', 'thought', 'through', 'out', 'that', 'Antipathies,', 'rather', 'sound', 'at', 'what', 'country', 'you', 'you’re', 'through', 'you', 'you', 'could', 'what', 'should', 'cat.)', 'at', 'you', 'you', 'catch', 'bat,', 'that’s

In [6]:
import time, requests, sys

class Index:
	def __init__(self):
		self.words = []
		self.index = {}

	def add_word(self, word):
		loc = len(self.words)
		self.words.append(word)

		for i in range(len(word)):
			for j in range(1, len(word) - i + 1):
				k = word[i:i+j]
				if k in self.index:
					self.index[k].append(loc)
				else:
					self.index[k] = [loc]

	def search(self, word):
		matches = [self.words[i] for i in self.index.get(word, [])]

		return matches


# util funcs

def get_words(large):
	if large:
		word_site = "https://www.mit.edu/~ecprice/wordlist.100000"
	else:
		word_site = "https://www.mit.edu/~ecprice/wordlist.10000"

	response = requests.get(word_site)
	words = response.content.splitlines()

	return [i.decode() for i in words]

def get_alice_words():
	word_site = 'https://www.gutenberg.org/files/11/11-0.txt'
	response = requests.get(word_site)

	words = response.text.replace('\n', '').split(' ')

	return words


def sizeof(obj):
    size = sys.getsizeof(obj)
    if isinstance(obj, dict): return size + sum(map(sizeof, obj.keys())) + sum(map(sizeof, obj.values()))
    if isinstance(obj, (list, tuple, set, frozenset)): return size + sum(map(sizeof, obj))
    return size

def trinum(n):
	return (n * (n + 1)) / 2


# benchmark an index vs string.find

def benchmark(index, words, word, n=100):
	length = 0

	time_a = time.time()
	for _ in range(n):
		length = len(index.search(word))
	time_a = (time.time() - time_a) / n

	time_b = time.time()
	for _ in range(n):
		length2 = len([i for i in words if i.find(word) != -1])
		if length2 != length:
			raise Exception("mismatching lengths: {} and {}".format(length, length2))
	time_b = (time.time() - time_b) / n

	return time_b / time_a


# run main

if __name__ == '__main__':
	words = get_words(False)
	#words = get_alice_words()

	start = time.time()
	index = Index()
	for i in words:
		index.add_word(i)
	
	print("indexing took {} with size of {}".format(time.time() - start, sizeof(index.index)))

	for test in ["so", "man", "sopololous", "tat", "tug"]:
		comparison = benchmark(index, words, test)
		print("{} performed by a factor of {}".format(test, comparison))

indexing took 0.1345658302307129 with size of 20320410
so performed by a factor of 67.44441696550018
man performed by a factor of 329.664570230608
sopololous performed by a factor of 2021.0297872340425
tat performed by a factor of 295.326352530541
tug performed by a factor of 1838.4420289855075


In [16]:
!pwd

/Users/rowenwitt/Desktop/takehome


### Pattern_Match Class
- Will be written to .py file and imported
- Was considering Rabin-Karp or Boyer-Moore-Horsepool
- Decided against above due to search object being list of strings
- Implementation is an index of each match in list
- Requires a reference list 


In [10]:
class PatternMatch():
    def __init__(self, body = []):
        self.table = {} # Make own hash table, with all functionality
        self.body = []
        
        self.add_to_table(body)
        

    def add_to_table(self, words):
        for word in words:
            n_pos = len(self.body)
            self.body.append(word)

            for k in range(len(word)):

                for j in range(k):
                    val = word[j:len(word) - (k - 1) + j]
                    if val in self.table:
                        self.table[val].add(n_pos)
                    else:
                        self.table[val] = {n_pos}
                    
                    
    def search_indexes(self, pattern):
        out = []
        for seq in pattern:
            if seq in self.table:
                out.append(self.table[seq])
            else:
                pass
            
        return out
    
    
    def search(self, pattern):
        indexes = [list(i) for i in self.search_indexes(pattern)]
        out = []
        for i in indexes:
            out.append([self.body[j] for j in i])
            
        return out
    
    
    def reindex(self, body = []):
        if body == []:
            body = self.body
        self.body = []
        self.table = {}
        self.add_to_table(body)
        
        
    def delete(self, values):
        vals = [i for i in self.body if i not in values]
        self.reindex(vals)


In [14]:
start = time.time()
CP = PatternMatch(test)
print('index time', time.time() - start)

start = time.time()
one = CP.search(['at', 'ou'])
print('search time 1', time.time() - start)
start = time.time()
two = CP.search(['in', 'mo'])
print('search time 2', time.time() - start)

index time 0.2635641098022461
search time 1 0.00055694580078125
search time 2 0.0003578662872314453


In [206]:
words = ["an", "dinosaur", "palindrome", "megaladon"]

In [207]:
v = CP.search(['meg', 'me'])
v

[['remember',
  'met',
  'Game,',
  'remembering',
  'jurymen',
  'summer',
  'replacement',
  '“Come,',
  'came',
  'named',
  'time',
  'exclaimed',
  'screamed',
  'me',
  'sometimes',
  'exclaimed',
  'renamed.',
  '“Come',
  'Some',
  'disclaimers',
  'something;',
  'some',
  'means',
  'something',
  'remembered',
  'time,”',
  'name,',
  'disclaimer',
  'name',
  'agreement',
  'game',
  'time,',
  'me',
  'Time',
  'agreement,',
  'me',
  '“Come',
  'agreement',
  'name',
  'something,',
  'disclaimer',
  'came,',
  'jurymen',
  'moment',
  'mean,”',
  'melancholy',
  'agreement',
  'me',
  'remember',
  'Time!”',
  'time',
  'something',
  'games',
  'seemed',
  'same',
  'time',
  'melancholy',
  'agreement,',
  'me',
  '“Same',
  'remember',
  'farmer,',
  'me',
  'me,”',
  'home',
  'time',
  'moment,',
  'meaning.',
  'moment',
  'me',
  'parchment',
  'screamed',
  'Time,',
  'time',
  'game’s',
  'moment',
  'come',
  'time).',
  'me',
  'commercial',
  'refreshments!”'

In [15]:
len(CP.body)

29592

# Part 2

Writes tests for your code using `pytest`

# Part 3

Imagine your job is to help build the text processing pipeline for a machine learning model. This model takes strings, processes them into vectors, and then classifies the text as relating to one of several categories (a multiclass classifier).

The raw data that is passed into the machine learning pipeline has strings that may include descriptions of many items, including multiple items that we are seeking to classify, as well as descriptions of items that are irrelevant to our classifier. These must be separated before being passed to classifier.

As the newest member of the team, your job is to get up to speed with the data as quickly as possible, and to find opportunities to improve the text processing pipeline. Perform an EDA on the following CSV to accomplish this. Be prepared to explain what insights you learned from the data, and what approaches you might use to improve the pipeline.

Perform the analysis in this notebook, below this prompt.

In [246]:
df = pd.read_csv('takehome.csv')

df

,id,text,added_datetime,yes_no,image_coordinates,image_url,original_id,page_number,parser_version,model_version,manual_override
0,462548,No plastic bags or pesticide containers .,2021-07-13 01:18:01.983,no,"{'originalSize': {'height': 1700, 'width': 220...",NaN,NaN,NaN,NaN,NaN,NaN
1,462549,No paint or aerosol cans .,2021-07-13 01:18:01.983,no,"{'originalSize': {'height': 1700, 'width': 220...",NaN,NaN,NaN,NaN,NaN,NaN
2,462550,Tin and aluminum food and drink cans .,2021-07-13 01:18:01.983,yes,"{'originalSize': {'height': 1700, 'width': 220...",NaN,NaN,NaN,NaN,NaN,NaN
3,462551,Corrugated and non corrugated . Includes cerea...,2021-07-13 01:18:01.983,yes,"{'originalSize': {'height': 1700, 'width': 220...",NaN,NaN,NaN,NaN,NaN,NaN
4,462552,"Includes all colors office paper , newspapers ...",2021-07-13 01:18:01.983,yes,"{'originalSize': {'height': 1700, 'width': 220...",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
995,305043,"aluminum, plastic containers",2020-11-16 15:27:38.157,yes,NaN,NaN,NaN,NaN,0.1.0,0.1.0,text
996,305044,cardboard,2020-11-16 15:27:38.157,yes,NaN,NaN,NaN,NaN,0.1.0,0.1.0,text
997,305045,"newspapers, and",2020-11-16 15:27:38.157,yes,NaN,NaN,NaN,NaN,0.1.0,0.1.0,text
998,305046,NaN,2020-11-16 15:27:38.157,yes,NaN,NaN,NaN,NaN,0.1.0,0.1.0,text
